## Extract Poses from Amass Dataset

In [21]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook
%matplotlib inline

import sys, os
import torch
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from tqdm import tqdm



from human_body_prior.tools.omni_tools import copy2cpu as c2c

os.environ['PYOPENGL_PLATFORM'] = 'egl'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Please remember to download the following subdataset from AMASS website: https://amass.is.tue.mpg.de/download.php. Note only download the <u>SMPL+H G</u> data.
* ACCD (ACCD)
* HDM05 (MPI_HDM05)
* TCDHands (TCD_handMocap)
* SFU (SFU)
* BMLmovi (BMLmovi)
* CMU (CMU)
* Mosh (MPI_mosh)
* EKUT (EKUT)
* KIT  (KIT)
* Eyes_Janpan_Dataset (Eyes_Janpan_Dataset)
* BMLhandball (BMLhandball)
* Transitions (Transitions_mocap)
* PosePrior (MPI_Limits)
* HumanEva (HumanEva)
* SSM (SSM_synced)
* DFaust (DFaust_67)
* TotalCapture (TotalCapture)
* BMLrub (BioMotionLab_NTroje)

### Unzip all datasets. In the bracket we give the name of the unzipped file folder. Please correct yours to the given names if they are not the same.

### Place all files under the directory **./amass_data/**. The directory structure shoud look like the following:  
./amass_data/  
./amass_data/ACCAD/  
./amass_data/BioMotionLab_NTroje/  
./amass_data/BMLhandball/  
./amass_data/BMLmovi/   
./amass_data/CMU/  
./amass_data/DFaust_67/  
./amass_data/EKUT/  
./amass_data/Eyes_Japan_Dataset/  
./amass_data/HumanEva/  
./amass_data/KIT/  
./amass_data/MPI_HDM05/  
./amass_data/MPI_Limits/  
./amass_data/MPI_mosh/  
./amass_data/SFU/  
./amass_data/SSM_synced/  
./amass_data/TCD_handMocap/  
./amass_data/TotalCapture/  
./amass_data/Transitions_mocap/  

**Please make sure the file path are correct, otherwise it can not succeed.**

In [22]:
# Choose the device to run the body model on.
comp_device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")

In [28]:
!pip install human_body_prior

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 101.7/101.7 kB ? eta 0:00:00
     ---------------------------------------- 42.7/42.7 kB ? eta 0:00:00
     --------------------------------------- 39.0/39.0 MB 40.9 MB/s eta 0:00:00
     --------------------------------------- 12.1/12.1 MB 50.4 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     -------------------------------------- 313.2/313.2 kB 6.4 MB/s eta 0:00:00
     ---------------------------------------- 1.4/1.4 MB 44.1 MB/s eta 0:00:00
     ------------------------------------- 694.7/694.7 kB 42.8 MB/s eta 0:00:00
     ---------------------------------------- 1.2/1.2 MB 75.2 MB/s eta 0:00:00
     ---------------------------------------- 1.9/1.9 MB 61.8 MB/s eta 0:00:00
     ------------------------------------- 178.9/178.9 kB 10.5 MB/s eta 0:00:00
     ------

In [30]:
from human_body_prior.body_model.body_model import BodyModel

male_bm_path = './body_models/smplh/male/model.npz'
male_dmpl_path = './body_models/dmpls/male/model.npz'

female_bm_path = './body_models/smplh/female/model.npz'
female_dmpl_path = './body_models/dmpls/female/model.npz'

num_betas = 10 # number of body parameters
num_dmpls = 8 # number of DMPL parameters

male_bm = BodyModel(bm_fname=male_bm_path, num_betas=num_betas, num_dmpls=num_dmpls, dmpl_fname=male_dmpl_path).to(comp_device)
faces = c2c(male_bm.f)

female_bm = BodyModel(bm_fname=female_bm_path, num_betas=num_betas, num_dmpls=num_dmpls, dmpl_fname=female_dmpl_path).to(comp_device)

This will take a few hours for all datasets, here we take one dataset as an example

To accelerate the process, you could run multiple scripts like this at one time.

The above code will extract poses from **AMASS** dataset, and put them under directory **"./pose_data"**

In [42]:
import os
from tqdm import tqdm
import time

# 1) 모든 파일 수집
paths = []
folders = []

for root, dirs, files in os.walk('./amass_data'):
    # root 예: ".\\amass_data\\ACCAD\\Female1General_c3d"
    folders.append(root)

    for name in files:
        full_path = os.path.join(root, name)
        paths.append(os.path.normpath(full_path))

print("Total files found:", len(paths))

# 2) dataset_name 을 정확히 추출 (amass_data 바로 아래 폴더)
dataset_names = []
for p in paths:
    parts = p.split(os.sep)
    # parts = ['.', 'amass_data', 'ACCAD', 'Female1General_c3d', 'file.npz']
    if len(parts) >= 3:
        dataset_name = parts[2]  # ACCAD, CMU, KIT etc
        if dataset_name not in dataset_names:
            dataset_names.append(dataset_name)

print("Dataset names detected:", dataset_names)

# 3) group_path 구하기
group_path = []
for name in dataset_names:
    subset = [p for p in paths if os.sep + name + os.sep in p]
    group_path.append(subset)
    print(f"{name}: {len(subset)} files")

# 4) pose_data 구조 만들기
save_root = './pose_data'

for p in paths:
    rel = os.path.relpath(p, './amass_data')
    save_dir = os.path.dirname(os.path.join(save_root, rel))
    os.makedirs(save_dir, exist_ok=True)

# 5) 전체 전처리
cur_count = 0
all_count = len(paths)

for subset in group_path:
    if len(subset) == 0:
        continue

    first_path = os.path.normpath(subset[0])
    parts = first_path.split(os.sep)
    dataset_name = parts[2]

    pbar = tqdm(subset)
    pbar.set_description(f"Processing: {dataset_name}")

    for path in pbar:
        rel = os.path.relpath(path, './amass_data')
        save_path = os.path.join(save_root, rel)

        root_no_ext, _ = os.path.splitext(save_path)
        save_path = root_no_ext + '.npy'

        fps = amass_to_pose(path, save_path)

    cur_count += len(subset)
    print(f'Processed / All (fps {fps}): {cur_count}/{all_count}')
    time.sleep(0.5)

Total files found: 14055
Dataset names detected: ['Female1General_c3d', 'Female1Gestures_c3d', 'Female1Running_c3d', 'Female1Walking_c3d', 'Male1General_c3d', 'Male1Running_c3d', 'Male1Walking_c3d', 'Male2General_c3d', 'Male2MartialArtsExtended_c3d', 'Male2MartialArtsKicks_c3d', 'Male2MartialArtsPunches_c3d', 'Male2MartialArtsStances_c3d', 'Male2Running_c3d', 'Male2Walking_c3d', 'MartialArtsWalksTurns_c3d', 's001', 's007', 's008', 's009', 's011', 'rub001', 'rub002', 'rub003', 'rub004', 'rub005', 'rub006', 'rub007', 'rub008', 'rub009', 'rub010', 'rub011', 'rub012', 'rub014', 'rub015', 'rub016', 'rub017', 'rub018', 'rub020', 'rub021', 'rub022', 'rub023', 'rub024', 'rub025', 'rub026', 'rub027', 'rub028', 'rub029', 'rub030', 'rub031', 'rub032', 'rub033', 'rub034', 'rub035', 'rub036', 'rub037', 'rub038', 'rub039', 'rub040', 'rub041', 'rub042', 'rub043', 'rub044', 'rub045', 'rub046', 'rub047', 'rub048', 'rub049', 'rub050', 'rub051', 'rub052', 'rub053', 'rub054', 'rub055', 'rub056', 'rub057',

Subject_33_F_MoSh: 22 files
Subject_34_F_MoSh: 22 files
Subject_35_F_MoSh: 22 files
Subject_36_F_MoSh: 23 files
Subject_37_F_MoSh: 22 files
Subject_38_F_MoSh: 22 files
Subject_39_F_MoSh: 22 files
Subject_3_F_MoSh: 22 files
Subject_40_F_MoSh: 22 files
Subject_41_F_MoSh: 21 files
Subject_42_F_MoSh: 22 files
Subject_43_F_MoSh: 22 files
Subject_44_F_MoSh: 22 files
Subject_45_F_MoSh: 22 files
Subject_46_F_MoSh: 22 files
Subject_47_F_MoSh: 22 files
Subject_48_F_MoSh: 22 files
Subject_4_F_MoSh: 22 files
Subject_50_F_MoSh: 22 files
Subject_51_F_MoSh: 22 files
Subject_52_F_MoSh: 22 files
Subject_53_F_MoSh: 22 files
Subject_54_F_MoSh: 22 files
Subject_55_F_MoSh: 22 files
Subject_56_F_MoSh: 22 files
Subject_57_F_MoSh: 21 files
Subject_58_F_MoSh: 22 files
Subject_59_F_MoSh: 22 files
Subject_5_F_MoSh: 22 files
Subject_60_F_MoSh: 22 files
Subject_61_F_MoSh: 22 files
Subject_62_F_MoSh: 22 files
Subject_63_F_MoSh: 22 files
Subject_64_F_MoSh: 22 files
Subject_65_F_MoSh: 22 files
Subject_66_F_MoSh: 22 f

Processing: Female1General_c3d: 100%|██████████████████████████████████████████████████| 17/17 [00:01<00:00, 15.82it/s]


Processed / All (fps 120.0): 17/14055


Processing: Female1Gestures_c3d: 100%|███████████████████████████████████████████████████| 6/6 [00:00<00:00,  6.39it/s]


...

Processed / All (fps 60.0): 13941/14055


Processing: s5: 100%|████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  2.51it/s]


Processed / All (fps 60.0): 13945/14055


Processing: mazen_c3d: 100%|█████████████████████████████████████████████████████████| 110/110 [00:07<00:00, 15.49it/s]


Processed / All (fps 120.0): 14055/14055


The source data from **HumanAct12** is already included in **"./pose_data"** in this repository. You need to **unzip** it right in this folder.

## Segment, Mirror and Relocate Motions

In [47]:
import os
import codecs as cs
import pandas as pd
import numpy as np
from tqdm import tqdm
from os.path import join as pjoin

def swap_left_right(data):
    assert len(data.shape) == 3 and data.shape[-1] == 3
    data = data.copy()
    data[..., 0] *= -1

    right_chain = [2, 5, 8, 11, 14, 17, 19, 21]
    left_chain  = [1, 4, 7, 10, 13, 16, 18, 20]

    left_hand_chain  = [22, 23, 24, 34, 35, 36, 25, 26, 27, 31, 32, 33, 28, 29, 30]
    right_hand_chain = [43, 44, 45, 46, 47, 48, 40, 41, 42, 37, 38, 39, 49, 50, 51]

    tmp = data[:, right_chain].copy()
    data[:, right_chain] = data[:, left_chain]
    data[:, left_chain]  = tmp

    if data.shape[1] > 24:
        tmp = data[:, right_hand_chain].copy()
        data[:, right_hand_chain] = data[:, left_hand_chain]
        data[:, left_hand_chain]  = tmp

    return data


index_path = './index.csv'
save_dir   = './joints'
index_file = pd.read_csv(index_path)

os.makedirs(save_dir, exist_ok=True)

total_amount = index_file.shape[0]
fps = 20

for i in tqdm(range(total_amount)):
    # Windows/Linux 호환 경로 변환
    source_path = os.path.normpath(index_file.loc[i]['source_path'])

    new_name = index_file.loc[i]['new_name']
    start_frame = index_file.loc[i]['start_frame']
    end_frame   = index_file.loc[i]['end_frame']

    # 데이터 로드
    data = np.load(source_path)

    # HumanAct12가 아니면 특별 케이스 처리
    if 'humanact12' not in source_path.lower():
        # dataset 이름 식별을 위해 소문자 비교
        sp = source_path.lower()

        if 'eyes_japan_dataset' in sp:
            data = data[3*fps:]
        if 'mpi_hdm05' in sp:
            data = data[3*fps:]
        if 'totalcapture' in sp:
            data = data[1*fps:]
        if 'mpi_limits' in sp:
            data = data[1*fps:]
        if 'transitions_mocap' in sp:
            data = data[int(0.5*fps):]

        # index.csv 지정 구간 자르기
        data = data[start_frame:end_frame]

        # 좌표계 반전
        data[..., 0] *= -1

    # Mirrored version 생성
    data_m = swap_left_right(data)

    # Windows/Linux 모두 사용 가능한 경로
    save_path      = pjoin(save_dir, new_name)
    save_path_mirr = pjoin(save_dir, 'M' + new_name)

    np.save(save_path, data)
    np.save(save_path_mirr, data_m)

100%|███████████████████████████████████████████████████████████████████████████| 14616/14616 [01:28<00:00, 165.65it/s]


In [46]:
import pandas as pd

index_file = pd.read_csv("./index.csv")

# 경로 안에 humanact12/humanact12 가 들어있으면 humanact12 하나만 남기기
index_file["source_path"] = index_file["source_path"].str.replace(
    "pose_data/humanact12/humanact12",
    "pose_data/humanact12"
)
index_file["source_path"] = index_file["source_path"].str.replace(
    r"pose_data\\humanact12\\humanact12",
    r"pose_data\\humanact12",
    regex=True
)

index_file.to_csv("./index.csv", index=False)
print("index.csv 경로 수정 완료")

index.csv 경로 수정 완료
